In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("/content/drive/MyDrive/df_train_not_encoded.csv",low_memory=False)
df_train.head()

,Id,OrgId,IncidentId,AlertId,DetectorId,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,...,RegistryValueName,ApplicationId,OAuthApplicationId,ResourceIdName,OSFamily,CountryCode,Day,Month,Hour,Year
0,180388628218,0,612,123247,7,6,InitialAccess,TruePositive,Ip,Related,...,635,2251,881,3586,5,31,4,6,6,2024
1,455266534868,88,326,210035,58,43,Exfiltration,FalsePositive,User,Impacted,...,635,2251,881,3586,5,242,14,6,3,2024
2,1056561957389,809,58352,712507,423,298,InitialAccess,FalsePositive,Url,Related,...,635,2251,881,3586,5,242,13,6,4,2024
3,1279900258736,92,32992,774301,2,2,CommandAndControl,BenignPositive,Url,Related,...,635,2251,881,3586,5,242,10,6,16,2024
4,214748368522,148,4359,188041,9,74,Execution,TruePositive,User,Impacted,...,635,2251,881,3586,5,242,15,6,1,2024


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9465497 entries, 0 to 9465496
Data columns (total 29 columns):
 #   Column              Dtype
---  ------              -----
 0   Id                  int64
 1   OrgId               int64
 2   IncidentId          int64
 3   AlertId             int64
 4   DetectorId          int64
 5   AlertTitle          int64
 6   Category            int64
 7   IncidentGrade       int64
 8   EntityType          int64
 9   EvidenceRole        int64
 10  DeviceId            int64
 11  Sha256              int64
 12  IpAddress           int64
 13  Url                 int64
 14  AccountSid          int64
 15  AccountUpn          int64
 16  DeviceName          int64
 17  NetworkMessageId    int64
 18  RegistryKey         int64
 19  RegistryValueName   int64
 20  ApplicationId       int64
 21  OAuthApplicationId  int64
 22  ResourceIdName      int64
 23  OSFamily            int64
 24  CountryCode         int64
 25  Day                 int64
 26  Month         

In [3]:
from sklearn.preprocessing import LabelEncoder

# List of categorical columns to encode
categorical_cols = ['Category', 'IncidentGrade', 'EntityType', 'EvidenceRole']

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder to each categorical column and replace the original columns
for column in categorical_cols:
    df_train[column] = le.fit_transform(df_train[column])

# Print the updated DataFrame to verify
print(df_train.head())


              Id  OrgId  IncidentId  AlertId  DetectorId  AlertTitle  \
0   180388628218      0         612   123247           7           6   
1   455266534868     88         326   210035          58          43   
2  1056561957389    809       58352   712507         423         298   
3  1279900258736     92       32992   774301           2           2   
4   214748368522    148        4359   188041           9          74   

   Category  IncidentGrade  EntityType  EvidenceRole  ...  RegistryValueName  \
0        10              2          15             1  ...                635   
1         7              1          32             0  ...                635   
2        10              1          31             1  ...                635   
3         1              0          31             1  ...                635   
4         6              2          32             0  ...                635   

   ApplicationId  OAuthApplicationId  ResourceIdName  OSFamily  CountryCode  \
0      

In [8]:
df_train.shape

(9465497, 29)

In [4]:
first_100k = df_train[:100000]
first_100k.head()


,Id,OrgId,IncidentId,AlertId,DetectorId,AlertTitle,Category,IncidentGrade,EntityType,EvidenceRole,...,RegistryValueName,ApplicationId,OAuthApplicationId,ResourceIdName,OSFamily,CountryCode,Day,Month,Hour,Year
0,180388628218,0,612,123247,7,6,10,2,15,1,...,635,2251,881,3586,5,31,4,6,6,2024
1,455266534868,88,326,210035,58,43,7,1,32,0,...,635,2251,881,3586,5,242,14,6,3,2024
2,1056561957389,809,58352,712507,423,298,10,1,31,1,...,635,2251,881,3586,5,242,13,6,4,2024
3,1279900258736,92,32992,774301,2,2,1,0,31,1,...,635,2251,881,3586,5,242,10,6,16,2024
4,214748368522,148,4359,188041,9,74,6,2,32,0,...,635,2251,881,3586,5,242,15,6,1,2024


In [5]:
from sklearn.model_selection import train_test_split
X= first_100k.drop('IncidentGrade',axis=1)
y= first_100k['IncidentGrade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier()

# Fit the model to training data
model.fit(X_train, y_train)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame to display feature importance
feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display top features
print(feature_importances.head(15))


             Feature  Importance
1              OrgId    0.216051
2         IncidentId    0.150860
4         DetectorId    0.109141
3            AlertId    0.102355
5         AlertTitle    0.086984
0                 Id    0.046250
6           Category    0.044912
24               Day    0.044524
26              Hour    0.035112
14        AccountUpn    0.027313
7         EntityType    0.025471
13        AccountSid    0.022602
11         IpAddress    0.015593
23       CountryCode    0.015421
16  NetworkMessageId    0.012491


In [8]:
# List of important features from the table
important_features = [
    'OrgId', 'IncidentId', 'DetectorId', 'AlertId', 'AlertTitle',
    'Id', 'Category', 'Day', 'Hour', 'AccountUpn',
    'EntityType', 'AccountSid', 'IpAddress', 'CountryCode', 'NetworkMessageId'
]

# Drop columns not in the list of important features
df_featured_train = df_train.drop(columns=[col for col in df_train.columns if col not in important_features])

# Print the resulting DataFrame
print(df_featured_train.head())


              Id  OrgId  IncidentId  AlertId  DetectorId  AlertTitle  \
0   180388628218      0         612   123247           7           6   
1   455266534868     88         326   210035          58          43   
2  1056561957389    809       58352   712507         423         298   
3  1279900258736     92       32992   774301           2           2   
4   214748368522    148        4359   188041           9          74   

   Category  EntityType  IpAddress  AccountSid  AccountUpn  NetworkMessageId  \
0        10          15         27      441377      673934            529644   
1         7          32     360606       22406       23032            529644   
2        10          31     360606      441377      673934            529644   
3         1          31     360606      441377      673934            529644   
4         6          32     360606         449         592            529644   

   CountryCode  Day  Hour  
0           31    4     6  
1          242   14     3  
2 

In [25]:
df_featured_train.to_csv("df_featured_train.csv", index=False)


In [16]:
x_featured = df_featured_train
y_featured=df_train["IncidentGrade"]

In [26]:
# Save y_featured as a CSV file directly
y_featured.to_csv('y_featured_series.csv', index=False)


In [17]:
x_featured.shape

(9465497, 15)

In [18]:
y_featured.shape

(9465497,)

In [19]:
X_train,X_test,y_train,y_test = train_test_split(x_featured,y_featured,test_size=0.2,random_state=42)

In [20]:
print("X_train :",X_train.shape)
print("X_test :",X_test.shape)
print("y_train :",y_train.shape)
print("y_test :",y_test.shape)

X_train : (7572397, 15)
X_test : (1893100, 15)
y_train : (7572397,)
y_test : (1893100,)


In [21]:
# Import  libraries
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [22]:
# Create a DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")  # You can change strategy based on your requirement

# Train the model
dummy_clf.fit(X_train, y_train)

# Make predictions
y_pred = dummy_clf.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.43478263166235276
Classification Report:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.43      1.00      0.61    823087
           1       0.00      0.00      0.00    406186
           2       0.00      0.00      0.00    663827

    accuracy                           0.43   1893100
   macro avg       0.14      0.33      0.20   1893100
weighted avg       0.19      0.43      0.26   1893100

Confusion Matrix:
[[823087      0      0]
 [406186      0      0]
 [663827      0      0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
